In [3]:
import pandas as pd
import pytesseract
from PIL import Image

# Load the image from file
img = Image.open("/Users/justinsheng/Library/Containers/com.tencent.xinWeChat/Data/Library/Application Support/com.tencent.xinWeChat/2.0b4.0.9/c84deda6a4ed60a9e3fa253bd79172f3/Message/MessageTemp/9e20f478899dc29eb19741386f9343c8/Image/2911700618581_.pic.jpg")

# Use tesseract to do OCR on the image
text = pytesseract.image_to_string(img)

# OCR gives us the entire text, so we will have to parse this text into a table.
# First, we split the text into lines
lines = text.split('\n')

# Helper function to parse a line into cells
def parse_line(line):
    # Remove unwanted characters and split by space to get each cell content
    return line.replace('%', '').strip().split()

# Parse the lines into a 2D list of data cells
data = [parse_line(line) for line in lines if line.strip()]

# The first line contains the headers, so separate it from the rest
headers = data[0]
# We also know the first header is 'Year', so we should remove it from the headers list
# and account for it when processing the data
headers = headers[1:]

# Now we process the rest of the lines to create the dataframe
# We initialize an empty list to hold all the rows for our dataframe
rows = []

# Each line represents a year and the subsequent inflation rates for each month
for line in data[1:]:
    year = line[0]
    # The rest of the line contains the inflation rates for each month
    for month, inflation in zip(headers, line[1:]):
        # We append a new row to our rows list
        rows.append([year, month, inflation])

# Convert the list of rows into a dataframe
df = pd.DataFrame(rows, columns=['Year', 'Month', 'Inflation'])

# We need to cast the Inflation column to float
df['Inflation'] = df['Inflation'].astype(float)

# Pivot the table to get the desired format
df_pivot = df.pivot(index=['Year', 'Month'], columns=[], values='Inflation').reset_index()

# Save to an Excel file
output_filepath = '/Users/justinsheng/Downloads/latest_inflation_data_canada_transformed.xlsx'
df_pivot.to_excel(output_filepath, index=False)

output_filepath


'/Users/justinsheng/Downloads/latest_inflation_data_canada_transformed.xlsx'